In [1]:
from az_common_funcs import *

2024-04-20 18:03:01,473 :: [INFO] :: Logger initialized WITHOUT file handler


In [2]:
## generator_loss.py
import torch
from torch import nn
from torchvision.models.vgg import vgg19

class TVLoss(nn.Module):
    def __init__(self, tv_loss_weight=1):
        super(TVLoss, self).__init__()
        self.tv_loss_weight = tv_loss_weight

    def forward(self, x):
        batch_size, c_x, h_x, w_x = x.size()
        count_h = self.tensor_size(x[:, :, 1:, :])
        count_w = self.tensor_size(x[:, :, :, 1:])
        h_tv = torch.pow((x[:, :, 1:, :] - x[:, :, :h_x - 1, :]), 2).sum()
        w_tv = torch.pow((x[:, :, :, 1:] - x[:, :, :, :w_x - 1]), 2).sum()
        return self.tv_loss_weight * 2 * (h_tv / count_h + w_tv / count_w) / batch_size

    @staticmethod
    def tensor_size(t):
        return t.size()[1] * t.size()[2] * t.size()[3]

class GeneratorLoss(nn.Module):
    def __init__(self):
        super(GeneratorLoss, self).__init__()
        vgg = vgg19(pretrained=True)
        loss_network = nn.Sequential(*list(vgg.features)[:37]).eval()
        for param in loss_network.parameters():
            param.requires_grad = False
        self.loss_network = loss_network
        self.mse_loss = nn.MSELoss()
        self.l1_loss = nn.L1Loss()
        self.tv_loss = TVLoss()

    def forward(self, out_labels, out_images, target_images):
        # Adversarial Loss
        adversarial_loss = torch.mean(1 - out_labels)
        # Perception Loss
        perception_loss = self.l1_loss(self.loss_network(out_images), self.loss_network(target_images))
        # Image Loss
        image_loss = self.mse_loss(out_images, target_images)
        # TV Loss
        tv_loss = self.tv_loss(out_images)
        return image_loss + 0.001 * adversarial_loss + 0.006 * perception_loss + 2e-8 * tv_loss

/home/azikre/aadil/venvs/season4/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/azikre/aadil/venvs/season4/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
## generator.py
import torch.nn as nn
import torch.nn.functional as F

class ganGenerator(nn.Module):
    def __init__(self):
        super(ganGenerator, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=9, stride=1, padding=4)
        self.prelu1 = nn.PReLU()
        # 16 blocks
        self.GRB1 = GeneratorResidualBlock()
        self.GRB2 = GeneratorResidualBlock()
        self.GRB3 = GeneratorResidualBlock()
        self.GRB4 = GeneratorResidualBlock()
        self.GRB5 = GeneratorResidualBlock()
        self.GRB6 = GeneratorResidualBlock()
        self.GRB7 = GeneratorResidualBlock()
        self.GRB8 = GeneratorResidualBlock()
        self.GRB9 = GeneratorResidualBlock()
        self.GRB10 = GeneratorResidualBlock()
        self.GRB11 = GeneratorResidualBlock()
        self.GRB12 = GeneratorResidualBlock()
        self.GRB13 = GeneratorResidualBlock()
        self.GRB14 = GeneratorResidualBlock()
        self.GRB15 = GeneratorResidualBlock()
        self.GRB16 = GeneratorResidualBlock()
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.pxlshuffle1 = nn.PixelShuffle(2)
        self.prelu2 = nn.PReLU()
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.pxlshuffle2 = nn.PixelShuffle(2)
        self.prelu3 = nn.PReLU()
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=3, kernel_size=9, stride=1, padding=4)

    def forward(self, x):
        x1 = self.conv1(x)
        x1 = self.prelu1(x1)

        x2 = self.GRB1(x1)
        x2 = self.GRB2(x2)
        x2 = self.GRB3(x2)
        x2 = self.GRB4(x2)
        x2 = self.GRB5(x2)
        x2 = self.GRB6(x2)
        x2 = self.GRB7(x2)
        x2 = self.GRB8(x2)
        x2 = self.GRB9(x2)
        x2 = self.GRB10(x2)
        x2 = self.GRB11(x2)
        x2 = self.GRB12(x2)
        x2 = self.GRB13(x2)
        x2 = self.GRB14(x2)
        x2 = self.GRB15(x2)
        x2 = self.GRB16(x2)

        x2 = self.conv2(x2)
        x2 = self.bn1(x2)
        x3 = x1 + x2

        x3 = self.conv3(x3)
        x3 = self.pxlshuffle1(x3)
        x3 = self.prelu2(x3)
        x3 = self.conv4(x3)
        x3 = self.pxlshuffle2(x3)
        x4 = self.prelu3(x3)

        x5 = self.conv5(x4)

        return x5

class GeneratorResidualBlock(nn.Module):
    def __init__(self):
        super(GeneratorResidualBlock, self).__init__()
        # convolution
        self.conv1 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        # batchnorm
        self.bn1 = nn.BatchNorm2d(64)
        # prelu
        self.prelu1 = nn.PReLU()
        #convolution
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        # batchnorm
        self.bn2 = nn.BatchNorm2d(64)
        # prelu
        self.prelu2 = nn.PReLU()
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.prelu1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.prelu2(out)
        return out + x

In [4]:
## discriminator.py
import torch.nn as nn

class ganDiscriminator(nn.Module):
    def __init__(self):
        super(ganDiscriminator, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1)
        self.lrelu1 = nn.LeakyReLU(0.2)

        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.lrelu2 = nn.LeakyReLU(0.2)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.lrelu3 = nn.LeakyReLU(0.2)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=2)
        self.bn4 = nn.BatchNorm2d(128)
        self.lrelu4 = nn.LeakyReLU(0.2)

        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.lrelu5 = nn.LeakyReLU(0.2)
        self.conv6 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=2)
        self.bn6 = nn.BatchNorm2d(256)
        self.lrelu6 = nn.LeakyReLU(0.2)

        self.conv7 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1)
        self.bn7 = nn.BatchNorm2d(512)
        self.lrelu7 = nn.LeakyReLU(0.2)
        self.conv8 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=2)
        self.bn8 = nn.BatchNorm2d(512)
        self.lrelu8 = nn.LeakyReLU(0.2)

        # self.flat = nn.Flatten()

        # self.dense9 = nn.Linear(in_features=60*124*512, out_features=1024, bias=True)
        # self.lrelu9 = nn.LeakyReLU()

        # self.dense10 = nn.Linear(in_features=1024, out_features=1, bias=True)
        # self.sigmoid10 = nn.Sigmoid()

        self.adaptive_pool_1 = nn.AdaptiveAvgPool2d(1)
        self.conv9 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=1)
        self.lrelu9 = nn.LeakyReLU(0.2)
        self.conv10 = nn.Conv2d(in_channels=1024, out_channels=1, kernel_size=1)
        self.sigmoid9 = nn.Sigmoid()

    def forward(self, x):
        x1 = self.conv1(x)
        x1 = self.lrelu1(x1)

        x2 = self.conv2(x1)
        x2 = self.bn2(x2)
        x2 = self.lrelu2(x2)

        x3 = self.conv3(x2)
        x3 = self.bn3(x3)
        x3 = self.lrelu3(x3)

        x4 = self.conv4(x3)
        x4 = self.bn4(x4)
        x4 = self.lrelu4(x4)

        x5 = self.conv5(x4)
        x5 = self.bn5(x5)
        x5 = self.lrelu5(x5)

        x6 = self.conv6(x5)
        x6 = self.bn6(x6)
        x6 = self.lrelu6(x6)

        x7 = self.conv7(x6)
        x7 = self.bn7(x7)
        x7 = self.lrelu7(x7)

        x8 = self.conv8(x7)
        x8 = self.bn8(x8)
        x8 = self.lrelu8(x8)

        x9 = self.adaptive_pool_1(x8)
        x9 = self.conv9(x9)
        x9 = self.lrelu9(x9)

        x10 = self.conv10(x9)
        x10 = self.sigmoid9(x10.view(x10.size()[0]))

        # x9 = self.flat(x8)
        # x10 = self.dense9(x9)
        # x10 = self.lrelu9(x10)
        # x11 = self.dense10(x10)
        # x11 = self.sigmoid10(x11)

        x11 = x10

        return x11

In [5]:
## data_utils.py
from PIL import Image
from torchvision.transforms import Compose, ToTensor, ToPILImage, CenterCrop, transforms, Resize
from torch.utils.data.dataset import Dataset

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in ['.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG'])

hr_target_size = (96, 96)

hr_transform = transforms.Compose([
    transforms.Lambda(lambda img: img.rotate(0) if img.size[0] > img.size[1] else img.rotate(90)),
    transforms.CenterCrop(hr_target_size),
    transforms.ToTensor()
])

lr_target_size = (96 // 4, 96 // 4)

lr_transform = transforms.Compose([
    transforms.Lambda(lambda img: img.rotate(0) if img.size[0] > img.size[1] else img.rotate(90)),
    transforms.CenterCrop(hr_target_size),
    transforms.Resize(lr_target_size, interpolation=Image.BICUBIC),
    transforms.ToTensor()
])


class Div2kTrainDataset(Dataset):
    def __init__(self, hr_base_dir):
        super(Div2kTrainDataset, self).__init__()

        self.hr_base_dir = hr_base_dir

        self.hr_image_filenames = [f'{self.hr_base_dir}/{i}.png' for i in range(0, 4900)]

        self.hr_transform = hr_transform
        self.lr_transform = lr_transform

    def __getitem__(self, index):
        hr_image = self.hr_transform(Image.open(self.hr_image_filenames[index]))
        lr_image = self.lr_transform(Image.open(self.hr_image_filenames[index]))
        return hr_image, lr_image

    def __len__(self):
        return len(self.hr_image_filenames)

class Div2kValDataset(Dataset):
    def __init__(self, hr_base_dir):
        super(Div2kValDataset, self).__init__()

        self.hr_base_dir = hr_base_dir

        self.hr_image_filenames = [f'{self.hr_base_dir}/{i}.png' for i in range(4900, 5000)]

        self.hr_transform = hr_transform
        self.lr_transform = lr_transform

    def __getitem__(self, index):
        hr_image = self.hr_transform(Image.open(self.hr_image_filenames[index]))
        lr_image = self.lr_transform(Image.open(self.hr_image_filenames[index]))
        return hr_image, lr_image

    def __len__(self):
        return len(self.hr_image_filenames)

def display_transform():
    return Compose([
        ToPILImage(),
        Resize(96),
        CenterCrop(96),
        ToTensor()
    ])

In [6]:
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

def compute_metrics_batch(hr_batch, sr_batch):
    n = hr_batch.shape[0]  # Number of elements in the batch
    psnr_values = np.zeros(n)
    ssim_values = np.zeros(n)
    
    for i in range(n):
        hr = hr_batch[i]
        sr = sr_batch[i]
        
        # Compute PSNR
        psnr_values[i] = psnr(hr, sr)
        
        # Compute SSIM
        ssim_values[i] = ssim(hr, sr, multichannel=True, channel_axis=0, data_range=sr.max() - sr.min())
    
    return psnr_values, ssim_values

In [7]:
from email_notifier import EmailNotifier
import dotenv
import os

dotenv.load_dotenv(".keys")

email_credentials = {'smtp_server':os.environ['email-smtp-server'],
                    'smtp_port':os.environ['email-smtp-port'],
                    'sender_email':os.environ['email-id'],
                    'sender_password':os.environ['email-pw']}

em = EmailNotifier(**email_credentials)

In [8]:
subject="Notification for SRGAN Training in College"
recipients="aadilzikre@gmail.com"

In [ ]:
## train.py
# aliased imports
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.utils as utils

# package imports no alias
import torch
import os
# import pytorch_ssim
import math

# Class/Fuction Imports
from tqdm import tqdm
from torch.autograd import Variable
from torch.utils.data import DataLoader

# Local Imports
# from generator import ganGenerator
# from discriminator import ganDiscriminator
# from generator_loss import GeneratorLoss
# from data_utils import Div2kValDataset, Div2kTrainDataset, display_transform

# filter warnings
import warnings
warnings.filterwarnings("ignore")

train_hr_loc = "data/train_1"
valid_hr_loc = "data/validation_1"

if __name__ == '__main__':

    NUM_EPOCHS = 4000
    START_AT = 2001
    LOAD_AT = START_AT - 1
    

    train_set = Div2kTrainDataset(train_hr_loc)
    val_set = Div2kValDataset(valid_hr_loc)
    train_loader = DataLoader(dataset=train_set, batch_size=64, shuffle=True)
    val_loader = DataLoader(dataset=val_set, batch_size=1, shuffle=False)


    netG = ganGenerator()
    if LOAD_AT >= 0:
        netG.load_state_dict(torch.load(f"epochs/netG_epoch_4_{LOAD_AT}.pth"), strict=False)
    print('# generator parameters:', sum(param.numel() for param in netG.parameters()))
    netD = ganDiscriminator()
    if LOAD_AT >= 0:
        netD.load_state_dict(torch.load(f"epochs/netD_epoch_4_{LOAD_AT}.pth"), strict=False)
    print('# discriminator parameters:', sum(param.numel() for param in netD.parameters()))

    generator_criterion = GeneratorLoss()

    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device_gen = torch.device("cuda:1")
    device_disc = torch.device("cuda:1")
    # if torch.cuda.is_available():
    #     netG.to(device)
    #     print("netG sent to cuda")
    #     netD.to(device)
    #     print("netD sent to cuda")
    #     generator_criterion.to(device)
    #     print("generator criterion sent to cuda")

    if torch.cuda.is_available():
        netG.to(device_gen)
        print("netG sent to cuda")
        netD.to(device_disc)
        print("netD sent to cuda")
        generator_criterion.to(device_gen)
        print("generator criterion sent to cuda")

    optimizerG = optim.Adam(netG.parameters(), lr=1e-5)
    optimizerD = optim.Adam(netD.parameters(), lr=1e-5)

    results = {'d_loss': [], 'g_loss': [], 'd_score': [], 'g_score': [], 'psnr': [], 'ssim': []}

    for epoch in range(START_AT, NUM_EPOCHS + 1):
        train_bar = tqdm(train_loader, ncols=150)
        running_results = {'batch_sizes': 0, 'd_loss': 0, 'g_loss': 0, 'd_score': 0, 'g_score': 0}

        netG.train()
        netD.train()

        for target, source in train_bar:
            g_update_first = True
            batch_size = target.size(0)
            running_results['batch_sizes'] += batch_size

            ############################
            # (1) Update D network: maximize D(x)-1-D(G(z))
            ###########################
            real_img = Variable(target)
            if torch.cuda.is_available():
                real_img = real_img.to(device_disc)
            z = Variable(source)
            if torch.cuda.is_available():
                z = z.to(device_gen)
            fake_img = netG(z).to(device_disc)

            netD.zero_grad()
            real_out = netD(real_img).mean()
            fake_out = netD(fake_img).mean()
            d_loss = 1 - real_out + fake_out
            d_loss.backward()
            optimizerD.step()

            ############################
            # (2) Update G network: minimize 1-D(G(z)) + Perception Loss + Image Loss + TV Loss
            ###########################
            netG.zero_grad()
            ## The two lines below are added to prevent runetime error in Google Colab ##
#             fake_img = netG(z).detach().to(device_disc)
            fake_img = netG(z).to(device_disc)
            fake_out = netD(fake_img).mean().to(device_gen)
            real_img = real_img.to(device_gen)
            fake_img = fake_img.to(device_gen)
            ##
            g_loss = generator_criterion(fake_out, fake_img, real_img)
            g_loss.backward()

#             fake_img = netG(z).detach().to(device_disc)
            fake_img = netG(z).to(device_disc)
            fake_out = netD(fake_img).mean()

            optimizerG.step()

            # loss for current batch before optimization
            running_results['g_loss'] += g_loss.item() * batch_size
            running_results['d_loss'] += d_loss.item() * batch_size
            running_results['d_score'] += real_out.item() * batch_size
            running_results['g_score'] += fake_out.item() * batch_size

            train_bar.set_description(desc='[%d/%d] Loss_D: %.6f Loss_G: %.4f D(x): %.6f D(G(z)): %.6f' % (
                epoch, NUM_EPOCHS, running_results['d_loss'] / running_results['batch_sizes'],
                running_results['g_loss'] / running_results['batch_sizes'],
                running_results['d_score'] / running_results['batch_sizes'],
                running_results['g_score'] / running_results['batch_sizes']))

        netG.eval()
        out_path = 'training_results/'
        if not os.path.exists(out_path):
            os.makedirs(out_path)

        with torch.no_grad():
            val_bar = tqdm(val_loader, ncols=150)
            validation_results = {'mse': 0, 'ssims': 0, 'psnr': 0, 'ssim': 0, 'batch_sizes': 0}
            val_images = []
            for val_hr, val_lr in val_bar:
                batch_size = val_lr.size(0)
                validation_results['batch_sizes'] += batch_size
                lr = val_lr
                hr = val_hr
                if torch.cuda.is_available():
                    lr = lr.to(device_gen)
                    hr = hr.to(device_gen)
                sr = netG(lr)

                batch_mse = ((sr - hr) ** 2).data.mean()
                validation_results['mse'] += batch_mse * batch_size
                # batch_ssim = pytorch_ssim.ssim(sr, hr).item()
                _, ssim_sr = compute_metrics_batch(hr.cpu().numpy(), sr.cpu().numpy())
                batch_ssim = ssim_sr / batch_size
                validation_results['ssims'] += batch_ssim * batch_size
                validation_results['psnr'] = 10 * math.log10((hr.max()**2) / (validation_results['mse'] / validation_results['batch_sizes']))
                validation_results['ssim'] = validation_results['ssims'] / validation_results['batch_sizes']
                val_bar.set_description(
                    desc='[converting LR images to SR images] PSNR: %.4f dB SSIM: %.4f' % (
                        validation_results['psnr'], validation_results['ssim']))

                val_images.extend(
                    [display_transform()(lr.squeeze(0)), display_transform()(hr.data.cpu().squeeze(0)),
                        display_transform()(sr.data.cpu().squeeze(0))])
            if epoch%100==0:
                val_images = torch.stack(val_images)
                val_images = torch.chunk(val_images, val_images.size(0) // 15)
                val_save_bar = tqdm(val_images, desc='[saving training results]')
                index = 1
                for image in val_save_bar:
                    image = utils.make_grid(image, nrow=3, padding=5)
                    utils.save_image(image, out_path + 'epoch_%d_index_%d.png' % (epoch, index), padding=5)
                    index += 1

        if not os.path.exists('epochs/'):
            os.makedirs('epochs/')
        # save model parameters
        if epoch%100==0:
            torch.save(netG.state_dict(), 'epochs/netG_epoch_%d_%d.pth' % (4, epoch))
            torch.save(netD.state_dict(), 'epochs/netD_epoch_%d_%d.pth' % (4, epoch))
            
        if epoch%100==0:
            training_res_string = '[%d/%d] Loss_D: %.6f Loss_G: %.4f D(x): %.6f D(G(z)): %.6f' % (
                epoch, NUM_EPOCHS, running_results['d_loss'] / running_results['batch_sizes'],
                running_results['g_loss'] / running_results['batch_sizes'],
                running_results['d_score'] / running_results['batch_sizes'],
                running_results['g_score'] / running_results['batch_sizes'])
            body = f"""
Epochs {epoch}/{NUM_EPOCHS} complete. 

Training Result String = {training_res_string}

Validation Results = {validation_results}
            """
            em.send_email(subject, body, recipients, attachments=[out_path + 'epoch_%d_index_%d.png' % (epoch, 20)])
            
        # save loss\scores\psnr\ssim
        results['d_loss'].append(running_results['d_loss'] / running_results['batch_sizes'])
        results['g_loss'].append(running_results['g_loss'] / running_results['batch_sizes'])
        results['d_score'].append(running_results['d_score'] / running_results['batch_sizes'])
        results['g_score'].append(running_results['g_score'] / running_results['batch_sizes'])
        results['psnr'].append(validation_results['psnr'])
        results['ssim'].append(validation_results['ssim'])

#         if epoch % 10 == 0 and epoch != 0:
#             out_path = '/kaggle/working/statistics/'
#             if not os.path.exists(out_path):
#                 os.makedirs(out_path)
#             data_frame = pd.DataFrame(
#                 data={'Loss_D': results['d_loss'], 'Loss_G': results['g_loss'], 'Score_D': results['d_score'],
#                         'Score_G': results['g_score'], 'PSNR': results['psnr'], 'SSIM': 0},
#                 index=range(1, epoch + 1))
#             data_frame.to_csv(out_path + 'srf_' + str(4) + '_train_results.csv', index_label='Epoch')

# generator parameters: 1549478
# discriminator parameters: 5215425
netG sent to cuda
netD sent to cuda
generator criterion sent to cuda


[2001/4000] Loss_D: 0.000003 Loss_G: 0.0042 D(x): 0.999999 D(G(z)): 0.000002: 100%|███████████████████████████████████| 77/77 [00:49<00:00,  1.55it/s]
[converting LR images to SR images] PSNR: 21.8725 dB SSIM: 0.6767: 100%|███████████████████████████████████████████| 100/100 [00:00<00:00, 112.88it/s]
[2002/4000] Loss_D: 0.000001 Loss_G: 0.0042 D(x): 0.999999 D(G(z)): 0.000000: 100%|███████████████████████████████████| 77/77 [00:49<00:00,  1.54it/s]
[converting LR images to SR images] PSNR: 21.9022 dB SSIM: 0.6773: 100%|███████████████████████████████████████████| 100/100 [00:00<00:00, 118.23it/s]
[2003/4000] Loss_D: 0.000002 Loss_G: 0.0041 D(x): 0.999998 D(G(z)): 0.000001:  40%|██████████████                     | 31/77 [00:20<00:30,  1.53it/s]